In [1]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings(action='ignore') 

In [80]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [148]:
test = pd.read_csv("C:/Users/user/Documents/Dacon/제주 특산물/test.csv")
train = pd.read_csv("C:/Users/user/Documents/Dacon/제주 특산물/train.csv")

In [149]:
test = pd.read_csv("C:/Users/user/Documents/Dacon/제주 특산물/test.csv")
test

,ID,timestamp,item,corporation,location
0,TG_A_J_20230304,2023-03-04,TG,A,J
1,TG_A_J_20230305,2023-03-05,TG,A,J
2,TG_A_J_20230306,2023-03-06,TG,A,J
3,TG_A_J_20230307,2023-03-07,TG,A,J
4,TG_A_J_20230308,2023-03-08,TG,A,J
...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J
1088,RD_F_J_20230328,2023-03-28,RD,F,J
1089,RD_F_J_20230329,2023-03-29,RD,F,J
1090,RD_F_J_20230330,2023-03-30,RD,F,J


In [150]:
test_df = test.copy()
train_df = train.copy()

In [151]:
train_df

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [152]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           59397 non-null  object 
 1   timestamp    59397 non-null  object 
 2   item         59397 non-null  object 
 3   corporation  59397 non-null  object 
 4   location     59397 non-null  object 
 5   supply(kg)   59397 non-null  float64
 6   price(원/kg)  59397 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.2+ MB


In [153]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           59397 non-null  object        
 1   timestamp    59397 non-null  datetime64[ns]
 2   item         59397 non-null  object        
 3   corporation  59397 non-null  object        
 4   location     59397 non-null  object        
 5   supply(kg)   59397 non-null  float64       
 6   price(원/kg)  59397 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 3.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092 entries, 0 to 1091
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           1092 non-null   object        
 1   timestamp    1092 non-null   datetime64[ns]
 2   item         1092 non-null   object        
 3   corporation  1092 non-nu

In [154]:
train_df['year'] = train_df['timestamp'].dt.year
train_df['month'] = train_df['timestamp'].dt.month
train_df['day'] = train_df['timestamp'].dt.day
test_df['year'] = test_df['timestamp'].dt.year
test_df['month'] = test_df['timestamp'].dt.month
test_df['day'] = test_df['timestamp'].dt.day

In [155]:
train_x = train_df.drop(columns = ['ID','timestamp','supply(kg)','price(원/kg)'])
train_y = train_df['price(원/kg)']
test_x = test_df.drop(columns = ['ID',"timestamp"])

In [156]:
train # 59397

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [157]:
train = train[train['price(원/kg)'] != 0]
train # 가격이 0원인 것 제외
# 23942

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
6,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0
7,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [158]:
train['item'] = train['item'].astype('category')
train['corporation'] = train['corporation'].astype('category')
train['location'] = train['location'].astype('category')
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
6,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0
7,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [159]:
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
6,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0
7,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [168]:
train['timestamp'] = pd.to_datetime(train['timestamp'])
# 계절로 바꾸는 건 어떨지
train['season'] = (train['timestamp'].dt.month % 12 + 2) // 3
season_mapping = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
train['season'] = train['season'].map(season_mapping)
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),season
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,Winter
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,Winter
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,Winter
6,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0,Winter
7,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0,Winter
...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,Winter
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,Winter
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,Winter
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,Winter


In [169]:
train['season'].value_counts()

Winter    10153
Spring     5492
Summer     2939
Fall       2845
Name: season, dtype: int64

In [163]:
# drop_first=True를 사용하는 경우:

# 다중 공선성 문제를 줄일 수 있습니다. 첫 번째 열이 다른 열들과 선형 의존 관계에 있어서 발생하는 문제를 방지할 수 있습니다.
# 변수 간의 관계를 더 명확하게 해줄 수 있습니다.
# drop_first=False를 사용하는 경우:

# 첫 번째 열이 의미 있는 정보를 제공하는 경우에는 유용할 수 있습니다.
# 모델이 해당 카테고리의 유무뿐만 아니라 각각의 값을 구분하는 데 도움이 필요한 경우에 사용할 수 있습니다.
# 데이터에 따라 첫 번째 카테고리가 중요한 정보를 가지고 있는 경우도 있습니다.
# 따라서 어떤 옵션을 선택할지는 사용자의 판단에 따라 다릅니다. 실험을 통해 모델의 성능을 평가하고 어떤 방식이 더 효과적인지 확인하는 것이 좋습니다.

In [170]:
# 범주형 변수를 원-핫 인코딩
train_encoded = pd.get_dummies(train, columns=['item', 'corporation', 'location','season'])
train_encoded
# drop_first 좋을 지 확인 해 봐야할듯...별로임 복잡해짐

,ID,timestamp,supply(kg),price(원/kg),item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,...,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S,season_Fall,season_Spring,season_Summer,season_Winter
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
6,TG_A_J_20190107,2019-01-07,44995.0,1474.0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
7,TG_A_J_20190108,2019-01-08,26975.0,1326.0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,452440.0,468.0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,1
59393,RD_F_J_20230228,2023-02-28,421980.0,531.0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,1
59394,RD_F_J_20230301,2023-03-01,382980.0,574.0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,1
59395,RD_F_J_20230302,2023-03-02,477220.0,523.0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,1


In [171]:
print(train_encoded.columns)

Index(['ID', 'timestamp', 'supply(kg)', 'price(원/kg)', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'corporation_F',
       'location_J', 'location_S', 'season_Fall', 'season_Spring',
       'season_Summer', 'season_Winter'],
      dtype='object')


In [172]:
# 필요한 열만 선택
train_encoded = train_encoded[['ID', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'corporation_F',
       'location_J', 'location_S', 'season_Fall', 'season_Spring',
       'season_Summer', 'season_Winter']]
train_encoded

,ID,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S,season_Fall,season_Spring,season_Summer,season_Winter
2,TG_A_J_20190103,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
3,TG_A_J_20190104,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
4,TG_A_J_20190105,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
6,TG_A_J_20190107,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
7,TG_A_J_20190108,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
59393,RD_F_J_20230228,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
59394,RD_F_J_20230301,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
59395,RD_F_J_20230302,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1


In [146]:
test
# test 데이터 작업시작

,ID,timestamp,item,corporation,location,season
0,TG_A_J_20230304,2023-03-04,TG,A,J,Winter
1,TG_A_J_20230305,2023-03-05,TG,A,J,Winter
2,TG_A_J_20230306,2023-03-06,TG,A,J,Winter
3,TG_A_J_20230307,2023-03-07,TG,A,J,Winter
4,TG_A_J_20230308,2023-03-08,TG,A,J,Winter
...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,Winter
1088,RD_F_J_20230328,2023-03-28,RD,F,J,Winter
1089,RD_F_J_20230329,2023-03-29,RD,F,J,Winter
1090,RD_F_J_20230330,2023-03-30,RD,F,J,Winter


In [173]:
test['timestamp'] = pd.to_datetime(test['timestamp'])
test['season'] = (test['timestamp'].dt.month % 12 + 2) // 3
season_mapping = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
test['season'] = test['season'].map(season_mapping)
test

,ID,timestamp,item,corporation,location,season
0,TG_A_J_20230304,2023-03-04,TG,A,J,Winter
1,TG_A_J_20230305,2023-03-05,TG,A,J,Winter
2,TG_A_J_20230306,2023-03-06,TG,A,J,Winter
3,TG_A_J_20230307,2023-03-07,TG,A,J,Winter
4,TG_A_J_20230308,2023-03-08,TG,A,J,Winter
...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,Winter
1088,RD_F_J_20230328,2023-03-28,RD,F,J,Winter
1089,RD_F_J_20230329,2023-03-29,RD,F,J,Winter
1090,RD_F_J_20230330,2023-03-30,RD,F,J,Winter


In [177]:
test['season'].value_counts()
# winter 밖에 없네?

Winter    1092
Name: season, dtype: int64

In [174]:
test_encoded = pd.get_dummies(test, columns=['item','corporation','location','season'])
test_encoded

,ID,timestamp,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S,season_Winter
0,TG_A_J_20230304,2023-03-04,0,0,0,0,1,1,0,0,0,0,0,1,0,1
1,TG_A_J_20230305,2023-03-05,0,0,0,0,1,1,0,0,0,0,0,1,0,1
2,TG_A_J_20230306,2023-03-06,0,0,0,0,1,1,0,0,0,0,0,1,0,1
3,TG_A_J_20230307,2023-03-07,0,0,0,0,1,1,0,0,0,0,0,1,0,1
4,TG_A_J_20230308,2023-03-08,0,0,0,0,1,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,0,0,0,1,0,0,0,0,0,0,1,1,0,1
1088,RD_F_J_20230328,2023-03-28,0,0,0,1,0,0,0,0,0,0,1,1,0,1
1089,RD_F_J_20230329,2023-03-29,0,0,0,1,0,0,0,0,0,0,1,1,0,1
1090,RD_F_J_20230330,2023-03-30,0,0,0,1,0,0,0,0,0,0,1,1,0,1


In [175]:
print(test_encoded.columns)

Index(['ID', 'timestamp', 'item_BC', 'item_CB', 'item_CR', 'item_RD',
       'item_TG', 'corporation_A', 'corporation_B', 'corporation_C',
       'corporation_D', 'corporation_E', 'corporation_F', 'location_J',
       'location_S', 'season_Winter'],
      dtype='object')


In [179]:
test_encoded = test_encoded[['ID', 'item_BC', 'item_CB', 'item_CR', 'item_RD',
       'item_TG', 'corporation_A', 'corporation_B', 'corporation_C',
       'corporation_D', 'corporation_E', 'corporation_F', 'location_J',
       'location_S', 'season_Winter']]
test_encoded

,ID,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S,season_Winter
0,TG_A_J_20230304,0,0,0,0,1,1,0,0,0,0,0,1,0,1
1,TG_A_J_20230305,0,0,0,0,1,1,0,0,0,0,0,1,0,1
2,TG_A_J_20230306,0,0,0,0,1,1,0,0,0,0,0,1,0,1
3,TG_A_J_20230307,0,0,0,0,1,1,0,0,0,0,0,1,0,1
4,TG_A_J_20230308,0,0,0,0,1,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,0,0,0,1,0,0,0,0,0,0,1,1,0,1
1088,RD_F_J_20230328,0,0,0,1,0,0,0,0,0,0,1,1,0,1
1089,RD_F_J_20230329,0,0,0,1,0,0,0,0,0,0,1,1,0,1
1090,RD_F_J_20230330,0,0,0,1,0,0,0,0,0,0,1,1,0,1


In [180]:
train_encoded

,ID,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S,season_Fall,season_Spring,season_Summer,season_Winter
2,TG_A_J_20190103,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
3,TG_A_J_20190104,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
4,TG_A_J_20190105,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
6,TG_A_J_20190107,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
7,TG_A_J_20190108,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
59393,RD_F_J_20230228,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
59394,RD_F_J_20230301,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1
59395,RD_F_J_20230302,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1


In [182]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense

# 내일 여기서 부터 다시 수정

In [ ]:
# 데이터 표준화 (예시로 표준화를 수행)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 딥러닝 모델 생성
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# 테스트 데이터로 예측
predictions = model.predict(X_test_scaled)

# 평가 지표 출력 (예시로 평균 제곱 오차 사용)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


In [36]:
train_X = train.iloc[:,:-1]
train_y = train.iloc[:,-1]
train_y

2        1728.0
3        1408.0
4        1250.0
6        1474.0
7        1326.0
          ...  
59392     468.0
59393     531.0
59394     574.0
59395     523.0
59396     529.0
Name: price(원/kg), Length: 23942, dtype: float64

In [37]:
test

,ID,timestamp,item,corporation,location
0,TG_A_J_20230304,2023-03-04,TG,A,J
1,TG_A_J_20230305,2023-03-05,TG,A,J
2,TG_A_J_20230306,2023-03-06,TG,A,J
3,TG_A_J_20230307,2023-03-07,TG,A,J
4,TG_A_J_20230308,2023-03-08,TG,A,J
...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J
1088,RD_F_J_20230328,2023-03-28,RD,F,J
1089,RD_F_J_20230329,2023-03-29,RD,F,J
1090,RD_F_J_20230330,2023-03-30,RD,F,J


# 딥러닝 시도

In [38]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [39]:
# 표준화 (Standardization)
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_scaled = scaler.transform(test)

ValueError: could not convert string to float: 'TG_A_J_20190103'

In [ ]:

# 딥러닝 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(train_X_scaled.shape[1],)),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_val_scaled, y_val))


# 머신러닝 시도

In [18]:
qual_col = ['item', 'corporation', 'location']

for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

In [16]:
rf = RandomForestRegressor()
rf_params = {"n_estimators":[10,30,50,70,100,200]}

In [17]:
rf.fit(train_x, train_y)

RandomForestRegressor()

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [19]:
grid_cf = GridSearchCV(rf,rf_params,scoring="neg_mean_squared_log_error",cv=5)

In [22]:
grid_cf.fit(train_x,train_y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'n_estimators': [10, 30, 50, 70, 100, 200]},
             scoring='neg_mean_squared_log_error')

In [23]:
grid_pred = grid_cf.predict(test_x)
grid_pred

array([3648.965, 3966.735,  508.495, ...,  435.33 ,  438.075,  437.28 ])

In [40]:
from sklearn.tree import DecisionTreeClassifier

In [43]:
dt = DecisionTreeClassifier()

In [45]:
dt_params = {"n_estimators" : [10,30,50,70,100,200]}
grid_cf = GridSearchCV(dt,dt_params,scoring='neg_mean_squared_log_error',cv=5)

In [47]:
grid_cf.fit(train_x,train_y)

ValueError: Invalid parameter 'n_estimators' for estimator DecisionTreeClassifier(). Valid parameters are: ['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'].

In [33]:
submission = pd.read_csv('./sample_submission.csv')
submission

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [48]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [77]:
knn_clf = KNeighborsClassifier()

param_grid = {'n_neighbors': [20],
              'weights': ['uniform'],
              'p': [2]}

grid_knn = GridSearchCV(knn_clf, param_grid, cv=5)

In [78]:
grid_knn.fit(train_x,train_y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [20], 'p': [2], 'weights': ['uniform']})

In [68]:
grid_knn.best_params_

{'n_neighbors': 20, 'p': 2, 'weights': 'uniform'}

In [79]:
grid_knn.best_score_

0.5914105530428453

In [76]:
knn_clf.fit(train_x, train_y)

KNeighborsClassifier()

In [81]:
knn_pred = knn_clf.predict(test_x)
knn_pred

array([1864.,    0.,    0., ...,  385.,  361.,    0.])

In [96]:
submission['answer'] = knn_pred

In [97]:
submission

,ID,answer
0,TG_A_J_20230304,1864.0
1,TG_A_J_20230305,0.0
2,TG_A_J_20230306,0.0
3,TG_A_J_20230307,900.0
4,TG_A_J_20230308,1595.0
...,...,...
1087,RD_F_J_20230327,0.0
1088,RD_F_J_20230328,425.0
1089,RD_F_J_20230329,385.0
1090,RD_F_J_20230330,361.0


In [87]:
submission.to_csv("./KNN_submission.csv",index = False)

In [88]:
ada_clf.fit(train_x, train_y)

AdaBoostClassifier(n_estimators=100)

In [102]:
test_x

,item,corporation,location,year,month,day
0,4,0,0,2023,3,4
1,4,0,0,2023,3,5
2,4,0,0,2023,3,6
3,4,0,0,2023,3,7
4,4,0,0,2023,3,8
...,...,...,...,...,...,...
1087,3,5,0,2023,3,27
1088,3,5,0,2023,3,28
1089,3,5,0,2023,3,29
1090,3,5,0,2023,3,30


In [103]:
ada_pred = ada_clf.predict(test_x)

In [104]:
ada_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [105]:
preds

array([3636.64, 4109.28,  401.82, ...,  439.31,  436.  ,  425.91])

In [106]:
submission

,ID,answer
0,TG_A_J_20230304,0.0
1,TG_A_J_20230305,0.0
2,TG_A_J_20230306,0.0
3,TG_A_J_20230307,0.0
4,TG_A_J_20230308,0.0
...,...,...
1087,RD_F_J_20230327,0.0
1088,RD_F_J_20230328,0.0
1089,RD_F_J_20230329,0.0
1090,RD_F_J_20230330,0.0


In [80]:
dt_clf.fit(train_x, train_y)

DecisionTreeClassifier()

In [81]:
dt_pred = dt_clf.predict(test_x)

In [124]:
submission['answer']=dt_pred

In [126]:
submission.to_csv('./DecisionTreeClassifer_submission.csv',index = False)

In [109]:
dt_pred # 의사 결정 트리

array([1864., 4423.,    0., ...,  385.,  361.,  370.])

In [120]:
preds # 랜덤 포레스트

array([3636.64, 4109.28,  401.82, ...,  439.31,  436.  ,  425.91])

In [121]:
knn_pred # KNN

array([1864.,    0.,    0., ...,  385.,  361.,    0.])

In [110]:
from sklearn.linear_model import LogisticRegression

In [113]:
pred = np.array([knn_pred,dt_pred,preds]) # (114, ) =>(4,114)
pred.shape

(3, 1092)

In [115]:
pred = np.transpose(pred)
pred.shape # (114,4)

(1092, 3)

In [118]:
lr_final = LogisticRegression()

In [122]:
lr_final.fit(dt_pred,train_y)

ValueError: Expected 2D array, got 1D array instead:
array=[1864. 4423.    0. ...  385.  361.  370.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics